In [ ]:
import pandas as pd
import evaluate
bleu = evaluate.load("bleu")

In [ ]:
set = 'gold'
df_trigger = pd.read_csv(f'../../data/generated/generated_trigger_{set}_one_shot.csv')
df_action = pd.read_csv(f'../../data/generated/generated_action_{set}_one_shot.csv')
df_full = pd.read_csv(f'../../data/generated/generated_recipe_{set}_one_shot.csv')

In [ ]:
total = len(df_trigger)
em_v = 0
predictions = []
references = []
trigger_pred = []
action_pred = []
trigger_ref = []
action_ref = []
em_t = 0
em_a = 0
for idx, row in df_full.iterrows():
    try:
        trigger_out, action_out = row['output'].split(' THEN ')
        action_out = 'THEN ' + action_out
        
        trigger_gen, action_gen = row['generated'].split(' THEN ')
        action_gen = 'THEN ' + action_gen
        
        output = row['output']
        generated = row['generated']
    except:
        total -= 1
        continue
    
    trigger_ref.append(trigger_out)
    action_ref.append(action_out)
    trigger_pred.append(trigger_gen)
    action_pred.append(action_gen)
    predictions.append(generated)  # BLEU expects tokenized sequences
    references.append(output)
    if output == generated:
        em_v += 1
    if trigger_out == trigger_gen:
        em_t += 1
    if action_out == action_gen:
        em_a += 1
        
bleu_score = bleu.compute(predictions=predictions, references=references)
bleu_trigger = bleu.compute(predictions=trigger_pred, references=trigger_ref)
bleu_action = bleu.compute(predictions=action_pred, references=action_ref)
result = {'em': em_v/total, 'em_t': em_t/total,'em_a': em_a/total, 'BLEU': bleu_score['bleu'], 'BLEU_trigger': bleu_trigger['bleu'], 'BLEU_action': bleu_action['bleu']}
print(result)

In [ ]:
import re
# Regular expression patterns
trigger_service_pattern = r"TRIGGER SERVICE:\s*([\w &]+)"
trigger_event_pattern = r"TRIGGER EVENT:\s*([\w &]+)"
action_service_pattern = r"ACTION SERVICE:\s*([\w &]+)"
action_event_pattern = r"ACTION EVENT:\s*([\w &]+)"

In [ ]:
total = len(df_trigger)
em_v = 0
predictions = []
references = []
trigger_pred = []
action_pred = []
trigger_ref = []
action_ref = []
em_t = 0
em_a = 0
for (trigger_idx, trigger_row), (action_idx, action_row) in zip(df_trigger.iterrows(), df_action.iterrows()):
    try:
        trigger_service_match = re.search(trigger_service_pattern, trigger_row['output'])
        trigger_event_match = re.search(trigger_event_pattern, trigger_row['output'])
        trigger_service = trigger_service_match.group(1).strip()
        trigger_event = trigger_event_match.group(1).strip()
        action_service_match = re.search(action_service_pattern, action_row['output'])
        action_event_match = re.search(action_event_pattern, action_row['output'])
        action_service = action_service_match.group(1).strip()
        action_event = action_event_match.group(1).strip()
        output = 'IF ' + trigger_service + ' ' + trigger_event + ' THEN ' + action_event + ' ' + action_service
        trigger_out = 'IF ' + trigger_service + ' ' + trigger_event
        action_out = 'THEN ' + action_event + ' ' + action_service
        
        trigger_service_match = re.search(trigger_service_pattern, str(trigger_row['generated']))
        trigger_event_match = re.search(trigger_event_pattern, str(trigger_row['generated']))
        trigger_service = trigger_service_match.group(1).strip()
        trigger_event = trigger_event_match.group(1).strip()
        action_service_match = re.search(action_service_pattern, str(action_row['generated']))
        action_event_match = re.search(action_event_pattern, str(action_row['generated']))
        action_service = action_service_match.group(1).strip()
        action_event = action_event_match.group(1).strip()
        generated = 'IF ' + trigger_service + ' ' + trigger_event + ' THEN ' + action_event + ' ' + action_service
        trigger_gen = 'IF ' + trigger_service + ' ' + trigger_event
        action_gen = 'THEN ' + action_event + ' ' + action_service
    except:
        total -= 1
        continue
    
    trigger_ref.append(trigger_out)
    action_ref.append(action_out)
    trigger_pred.append(trigger_gen)
    action_pred.append(action_gen)
    predictions.append(generated)  # BLEU expects tokenized sequences
    references.append(output)
    if output == generated:
        em_v += 1
    if trigger_out == trigger_gen:
        em_t += 1
    if action_out == action_gen:
        em_a += 1
        
bleu_score = bleu.compute(predictions=predictions, references=references)
bleu_trigger = bleu.compute(predictions=trigger_pred, references=trigger_ref)
bleu_action = bleu.compute(predictions=action_pred, references=action_ref)
result = {'em': em_v/total, 'em_t': em_t/total,'em_a': em_a/total, 'BLEU': bleu_score['bleu'], 'BLEU_trigger': bleu_trigger['bleu'], 'BLEU_action': bleu_action['bleu']}
print(result)

In [ ]:
total = len(df_trigger)
em_v = 0
predictions = []
references = []
for index, row in df_full.iterrows():
    output = str(row['output'])
    generated = str(row['generated'])
    if output == generated:
        em_v += 1
    predictions.append(generated)  # BLEU expects tokenized sequences
    references.append([output])


bleu_score = bleu.compute(predictions=predictions, references=references)

result = {'em': em_v/total, 'BLEU': bleu_score['bleu']}
print(result)